In [1]:
from clustering.clustering import get_clusterer
from data.read_data import *
from dimensionality_reduction.embedding import get_embedder
from experiment import Experiment

import argparse
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Clustering results with infinite gaussian with dirichlet process prior

In [3]:
twosides = get_twosides_meddra(pickle=False)
bayesian_results = pd.read_csv("../results/bayesian_gaussian/results.csv")
#kmeans_meddra = match_meddra(bayesian_results, twosides)

#(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
#.sort_values(ascending=False)\
#.sort_index(level=0, sort_remaining=False)\
#.groupby("cluster").nlargest(8)

<img src="../results/bayesian_gaussian/embedded_clusters.png">

In [8]:
twosides = get_twosides_meddra(pickle=False)
kmeans10 = pd.read_csv("../results/kmeans_10/kmeans_10_results.csv")

#(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
#.sort_values(ascending=False)\
#.sort_index(level=0, sort_remaining=False)\
#.groupby("cluster").nlargest(8)

In [4]:
def intersect2D(a, b):
  """
  Find row intersection between 2D numpy arrays, a and b.
  Returns another numpy array with shared rows
  """
  return np.array([x for x in set(tuple(x) for x in a) & set(tuple(x) for x in b)])

In [5]:
def cluster_intersection(results1, results2):
    size = []
    assignments = []
    for i in range(len(results1["cluster"].unique())):
        size.append(results1[results1["cluster"] == i].count()["cluster"])
        assignments.append(results1[results1["cluster"] == i][results1.columns[0:2]].values)
    
    assignments_ordered = []
    ranks = np.argsort(size)
    for j in range(len(size)):
        assignments_ordered.append(assignments[ranks[j]])
    
    size2 = []
    assignments2 = []
    for i in range(len(results2["cluster"].unique())):
        size2.append(results2[results2["cluster"] == i].count()["cluster"])
        assignments2.append(results2[results2["cluster"] == i][results2.columns[0:2]].values)
    
    assignments_ordered2 = []
    ranks2 = np.argsort(size2)
    for j in range(len(size2)):
        assignments_ordered2.append(assignments2[ranks2[j]])
    
    mat = np.zeros((len(assignments_ordered), len(assignments_ordered2)))
    
    for i in range(len(assignments_ordered)):
        for j in range(len(assignments_ordered2)):
            mat[i, j] = len(intersect2D(assignments_ordered[i], assignments_ordered2[j]))
    
    v1 = np.array(size)
    v2 = np.array(size2)
    
    return mat/v1[:, None], mat/v2[None, :]

In [11]:
m1, m2 = cluster_intersection(bayesian_results, kmeans10)

In [12]:
m1

array([[0.01102603, 0.01145482, 0.01212864, 0.01500766, 0.01819296,
        0.01794793, 0.02003063, 0.02254211, 0.02425727, 0.0261562 ],
       [0.01402584, 0.02278137, 0.02014621, 0.02575655, 0.03230194,
        0.02261136, 0.02805168, 0.03247195, 0.03204692, 0.03621217],
       [0.01829238, 0.02167623, 0.01951841, 0.02613898, 0.0277083 ,
        0.02927762, 0.02957187, 0.03874258, 0.0400667 , 0.04757001],
       [0.06678911, 0.10118551, 0.10786442, 0.13541493, 0.11036901,
        0.13174153, 0.15478377, 0.2055435 , 0.19034897, 0.2319252 ],
       [0.09876971, 0.11505805, 0.11956333, 0.16149714, 0.17830532,
        0.19996534, 0.20637671, 0.22890314, 0.26615838, 0.30220066]])

In [13]:
m2

array([[0.03197158, 0.03126567, 0.02099237, 0.03839524, 0.07865466,
        0.03761232, 0.09258211, 0.08375057, 0.07341491, 0.05384615],
       [0.02930728, 0.04480856, 0.02512723, 0.04748472, 0.10063559,
        0.03414634, 0.09343148, 0.08693673, 0.06989247, 0.05372005],
       [0.06625222, 0.07390069, 0.04219678, 0.08352923, 0.14962924,
        0.07663671, 0.1707248 , 0.17979062, 0.15146459, 0.1223203 ],
       [0.07104796, 0.10132085, 0.06849025, 0.12709607, 0.17505297,
        0.1012837 , 0.26245753, 0.28015476, 0.21134594, 0.17515763],
       [0.10124334, 0.11101822, 0.07315522, 0.14605861, 0.27251059,
        0.14813864, 0.33720272, 0.30063723, 0.28476085, 0.21992434]])

In [81]:
intersect2D(assignments_ordered[1], assignments_ordered[4])

array([], dtype=float64)

Clustering results with kmeans and 10 centers

In [86]:
len(intersect2D(assignments_ordered[0], assignments_ordered2[0]))

180

Gaussian mixture model with 5 components clustering

In [20]:
twosides = get_twosides_meddra(pickle=False)
kmeans_results = pd.read_csv("../results/gaussian_mixture/gmm_10_results.csv")
kmeans_meddra = match_meddra(kmeans_results, twosides)

(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
.sort_values(ascending=False)\
.sort_index(level=0, sort_remaining=False)\
.groupby("cluster").nlargest(5)

cluster  cluster  soc_term                                            
0        0        Nervous system disorders                                0.099148
                  Gastrointestinal disorders                              0.075927
                  General disorders and administration site conditions    0.075538
                  Vascular disorders                                      0.071402
                  Respiratory, thoracic and mediastinal disorders         0.064005
1        1        Nervous system disorders                                0.097493
                  Gastrointestinal disorders                              0.076335
                  General disorders and administration site conditions    0.073574
                  Vascular disorders                                      0.071943
                  Respiratory, thoracic and mediastinal disorders         0.063548
2        2        Nervous system disorders                                0.097830
                